This notebook reflects the comparison of ML-SABRE to SABRE using:
- Qiskit version: 2.2.3
- Sabre with `layout_trials = 2500`

as done originally with `sabre_layout = SabreLayout(coupling_map = device, seed = random_seed, layout_trials=number_of_trial, skip_routing=False)`

In [ ]:
import qiskit

print("Qiskit version:", qiskit.__version__)

Qiskit version: 2.2.3


Import results from ML Sabre and SABRE

In [7]:
filename = "results/comparison_circuits_2500_trials.csv"

Setup for analysis of results

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Tuple, Optional


def _geo_mean(x: pd.Series) -> float:
    x = pd.to_numeric(x, errors="coerce")
    x = x[(x > 0) & np.isfinite(x)]
    if len(x) == 0:
        return np.nan
    return float(np.exp(np.log(x).mean()))


def make_results_table_with_geo(
    csv_file: str,
) :
    df = pd.read_csv(csv_file)
    circuit = df["qasm_file"].astype(str).apply(lambda p: Path(p).name.replace(".qasm", ""))

    out = pd.DataFrame({
        ("", "Circuit"): circuit,
        ("SABRE (default)", "SWAP"): df["sabre_swaps"],
        ("SABRE (default)", "Depth"): df["sabre_depth"],
        ("SABRE (default)", "2Q Depth"): df["sabre_2q_depth"],
        ("SABRE (default)", "Runtime (s)"): df["sabre_time_s"],
        ("ML-SABRE (10 cycles)", "SWAP"): df["multilevel_swaps"],
        ("ML-SABRE (10 cycles)", "Depth"): df["multilevel_depth"],
        ("ML-SABRE (10 cycles)", "2Q Depth"): df["multilevel_2q_depth"],
        ("ML-SABRE (10 cycles)", "Runtime (s)"): df["multilevel_time_s"],
    }).sort_values(("", "Circuit")).reset_index(drop=True)

    # --- Geo. Ratio row values (keep as floats) ---
    def ratio_gmean(s_col, m_col):
        r = df[s_col] / df[m_col].replace(0, np.nan)
        return _geo_mean(r)

    geo_row = {
        ("", "Circuit"): "Geo. Ratio",
        ("SABRE (default)", "SWAP"): ratio_gmean("sabre_swaps", "multilevel_swaps"),
        ("SABRE (default)", "Depth"): ratio_gmean("sabre_depth", "multilevel_depth"),
        ("SABRE (default)", "2Q Depth"): ratio_gmean("sabre_2q_depth", "multilevel_2q_depth"),
        ("SABRE (default)", "Runtime (s)"): ratio_gmean("sabre_time_s", "multilevel_time_s"),
        ("ML-SABRE (10 cycles)", "SWAP"): 1.0,
        ("ML-SABRE (10 cycles)", "Depth"): 1.0,
        ("ML-SABRE (10 cycles)", "2Q Depth"): 1.0,
        ("ML-SABRE (10 cycles)", "Runtime (s)"): 1.0,
    }
    out = pd.concat([out, pd.DataFrame([geo_row])], ignore_index=True)

    # --- Formatters for normal rows ---
    def int_fmt(x):
        try:
            return f"{int(round(float(x)))}"
        except Exception:
            return ""

    def two_dec_fmt(x):
        try:
            return f"{float(x):.2f}"
        except Exception:
            return ""

    fmt = {}
    for col in out.columns:
        if col[1] == "Runtime (s)":
            fmt[col] = two_dec_fmt       # runtimes -> 2 decimals
        elif col[1] != "Circuit":
            fmt[col] = int_fmt           # other metrics -> integers

    styler = out.style.format(fmt)

    # --- Override formatting ONLY for the Geo. Ratio row (2 decimals everywhere except Circuit) ---
    geo_idx = out.index[out[("", "Circuit")] == "Geo. Ratio"]
    if len(geo_idx):
        geo_idx = [int(geo_idx[0])]
        cols_to_override = [c for c in out.columns if c[1] not in ("Circuit", "Runtime (s)")]
        idx = pd.IndexSlice
        styler = styler.format(two_dec_fmt, subset=idx[geo_idx, cols_to_override])
        # runtime columns already 2 decimals; leave as is

    return out, styler



Visualize the results as a table with geometric means

In [6]:
df_table, styled = make_results_table_with_geo(filename)
styled  